In [57]:
import pandas as pd

real_estate_df = pd.read_csv("Data/refined-real-estate.csv").drop(
    columns=["Unnamed: 0"]
)
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224576 entries, 0 to 224575
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   접수연도              224576 non-null  int64  
 1   자치구명              224576 non-null  object 
 2   법정동명              224576 non-null  object 
 3   본번                224576 non-null  int64  
 4   부번                224576 non-null  int64  
 5   건물명               224576 non-null  object 
 6   물건금액(만원)          224576 non-null  int64  
 7   건물면적(㎡)           224576 non-null  float64
 8   층                 224576 non-null  int64  
 9   건축년도              224576 non-null  int64  
 10  건물용도              224576 non-null  object 
 11  신고구분              224576 non-null  object 
 12  신고한 개업공인중개사 시군구명  224576 non-null  object 
 13  계약연도              224576 non-null  int64  
 14  계약월               224576 non-null  int64  
 15  계약일               224576 non-null  int64  
dtypes: float64(1), int64

In [58]:
address_df = pd.read_csv("Data/address-to-geo.csv").set_index("address")
address_df

,lon,lat
address,,
서울특별시 강동구 명일동 41,127.151841,37.549489
서울특별시 양천구 신정동 333,126.873597,37.517713
서울특별시 구로구 개봉동 261-10,126.856696,37.490298
서울특별시 강동구 길동 320-2,127.142900,37.544728
서울특별시 동작구 상도동 264-124,126.936804,37.498164
...,...,...
서울특별시 은평구 응암동 97-37,126.925406,37.599768
서울특별시 강서구 화곡동 408-60,126.845355,37.536824
서울특별시 마포구 망원동 432-60,126.901823,37.558830


In [59]:
lons = []
lats = []
for district_name, dong_name, main_number, sub_number in zip(
    real_estate_df["자치구명"],
    real_estate_df["법정동명"],
    real_estate_df["본번"],
    real_estate_df["부번"],
):
    address_name = (
        f"서울특별시 {district_name} {dong_name} {main_number}"
        if sub_number == 0
        else f"서울특별시 {district_name} {dong_name} {main_number}-{sub_number}"
    )
    if address_name in address_df.index:
        lons.append(address_df.loc[address_name].lon)
        lats.append(address_df.loc[address_name].lat)
    else:
        lons.append(pd.NA)
        lats.append(pd.NA)
real_estate_df["위도"] = pd.DataFrame(lats)
real_estate_df["경도"] = pd.DataFrame(lons)
real_estate_df.dropna(inplace=True)
real_estate_df["위도"] = real_estate_df["위도"].astype(float)
real_estate_df["경도"] = real_estate_df["경도"].astype(float)

real_estate_df

,접수연도,자치구명,법정동명,본번,부번,건물명,물건금액(만원),건물면적(㎡),층,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,계약연도,계약월,계약일,위도,경도
0,2025,도봉구,방학동,638,10,한신빌라(638-10),14800,37.66,4,1994,연립다세대,중개거래,서울 도봉구,2025,1,17,37.667592,127.036982
1,2025,강서구,등촌동,628,13,현대프린스텔,10300,26.16,6,1999,오피스텔,중개거래,서울 강서구,2025,1,17,37.560088,126.856361
2,2025,관악구,신림동,1639,51,푸리마타운,12700,18.70,8,2014,오피스텔,중개거래,서울 관악구,2025,1,17,37.483391,126.927468
3,2025,구로구,고척동,339,0,고척파크푸르지오,84000,59.89,9,2009,아파트,중개거래,"서울 구로구, 서울 양천구",2025,1,17,37.507611,126.858113
4,2025,광진구,중곡동,73,70,욱현하이브(73-70),19800,44.96,3,2009,연립다세대,중개거래,서울 동작구,2025,1,17,37.558748,127.095448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224560,2022,강서구,내발산동,704,19,아이디움102동,25400,36.20,2,2014,연립다세대,신고없음,없음,2020,4,3,37.552011,126.837554
224570,2022,서초구,서초동,1315,0,진흥아파트,155000,101.32,14,1979,아파트,신고없음,없음,2018,7,21,37.497346,127.022614
224573,2022,성북구,정릉동,260,10,진안빌라A(260-10),35000,57.95,2,1991,연립다세대,신고없음,없음,2016,6,20,37.613738,127.009158
224574,2022,성북구,정릉동,260,10,진안빌라A(260-10),35000,57.95,2,1991,연립다세대,신고없음,없음,2016,4,26,37.613738,127.009158


In [ ]:
real_estate_df.dropna(inplace=True)
real_estate_df.to_csv("Data/refined-real-estate2.csv")